In [1]:
input_assembly = 'hdfs:///files/salmonella/assembly_reconstructed'
input_reference = 'hdfs:///files/salmonella/assembledASM694v2'

In [2]:
import findspark
findspark.init()
findspark.find()

'/usr/local/spark/python/pyspark'

In [3]:
from pyspark.sql import SparkSession

spark = (SparkSession
         .builder
         .master("yarn")
         .appName("python-testing")
         .config("spark.executor.instances", 16)
         .config("spark.executor.memory", "1536m")
         .getOrCreate())

sc = spark.sparkContext
sc

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-04-25 16:04:27,613 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2022-04-25 16:04:38,845 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


<SparkContext master=yarn appName=python-testing>

In [4]:
assembly = sc.pickleFile(input_assembly).repartition(32).cache()
assembly.take(1)

[(100080, ('T', 0.9411764705882353, 16, [('T', 16), ('A', 1)]))]

In [5]:
def preprocess(raw, segmentLength):
    indexed = raw.zipWithIndex().map(lambda x: (x[1], x[0]))
    atomized = indexed.flatMap(lambda x: [(x[0] * segmentLength + i, v) for i, v in enumerate(x[1])])
    return atomized

In [6]:
reference = preprocess(sc.textFile(input_reference).filter(lambda x: not x.startswith('>')), 80).repartition(32).cache()
reference.take(1)

[(80, 'G')]

In [7]:
assembly.lookup(80)

[('G', 1.0, 25, [('G', 25)])]

In [8]:
joined = reference.join(assembly)
joined.take(1)

[(3882432, ('A', ('A', 1.0, 16, [('A', 16)])))]

In [9]:
#joined.repartition(1).saveAsPickleFile('/home/ubuntu/assembly_joined')

In [10]:
total_basepairs = reference.count()
total_basepairs

4951383

In [11]:
joined.count()

4950572

In [12]:
matches = joined.mapValues(lambda x: (x[0] == x[1][0], x)).filter(lambda x: x[1][0])
equal_basepairs = matches.count()
equal_basepairs

4949744

In [13]:
equal_basepairs / total_basepairs

0.9996689813734869

In [14]:
not_matched = joined.mapValues(lambda x: (x[0] == x[1][0], x)).filter(lambda x: not x[1][0])
not_matched.count()

828

In [15]:
not_matched.take(10)

[(319232, (False, ('T', ('A', 1.0, 1, [('A', 1)])))),
 (832832, (False, ('C', ('T', 0.90625, 29, [('T', 29), ('C', 3)])))),
 (1128512,
  (False, ('G', ('T', 0.5882352941176471, 10, [('T', 10), ('G', 7)])))),
 (4857600, (False, ('G', ('C', 1.0, 30, [('C', 30)])))),
 (2896000, (False, ('A', ('C', 0.5, 2, [('C', 2), ('A', 2)])))),
 (2745664, (False, ('G', ('A', 0.5454545454545454, 6, [('A', 6), ('G', 5)])))),
 (830720,
  (False, ('A', ('G', 0.7391304347826086, 17, [('G', 17), ('A', 6)])))),
 (831680,
  (False, ('T', ('C', 0.5483870967741935, 17, [('C', 17), ('T', 14)])))),
 (4857664, (False, ('G', ('T', 1.0, 31, [('T', 31)])))),
 (830208,
  (False,
   ('T', ('C', 0.6363636363636364, 7, [('C', 7), ('T', 3), ('G', 1)]))))]

In [16]:
min_reads_threshold = 5
min_confidence_threshold = 0.7
confident_bases = joined.filter(lambda x: x[1][1][2] >= min_reads_threshold and x[1][1][1] >= min_confidence_threshold)
confident_bases.take(10)

[(4857536, ('C', ('C', 1.0, 20, [('C', 20)]))),
 (4858176, ('T', ('T', 1.0, 21, [('T', 21)]))),
 (4858496, ('T', ('T', 1.0, 23, [('T', 23)]))),
 (4858816, ('T', ('T', 1.0, 37, [('T', 37)]))),
 (4859136, ('A', ('A', 1.0, 43, [('A', 43)]))),
 (4859456, ('A', ('A', 1.0, 28, [('A', 28)]))),
 (4859776, ('A', ('A', 1.0, 27, [('A', 27)]))),
 (4860096, ('A', ('A', 0.9705882352941176, 33, [('A', 33), ('C', 1)]))),
 (4860416, ('C', ('C', 1.0, 46, [('C', 46)]))),
 (4860736, ('T', ('T', 0.9791666666666666, 47, [('T', 47), ('A', 1)])))]

In [17]:
confident_matches = confident_bases.mapValues(lambda x: (x[0] == x[1][0], *x))
confident_matches.take(15)

[(64, (True, 'A', ('A', 1.0, 24, [('A', 24)]))),
 (384, (True, 'T', ('T', 1.0, 37, [('T', 37)]))),
 (704, (True, 'T', ('T', 0.9523809523809523, 20, [('T', 20), ('A', 1)]))),
 (1024, (True, 'A', ('A', 1.0, 18, [('A', 18)]))),
 (1344, (True, 'G', ('G', 1.0, 23, [('G', 23)]))),
 (1664, (True, 'G', ('G', 1.0, 27, [('G', 27)]))),
 (1984, (True, 'T', ('T', 1.0, 28, [('T', 28)]))),
 (2304, (True, 'A', ('A', 0.9545454545454546, 21, [('A', 21), ('C', 1)]))),
 (2624, (True, 'T', ('T', 1.0, 27, [('T', 27)]))),
 (2944, (True, 'C', ('C', 0.9705882352941176, 33, [('C', 33), ('A', 1)]))),
 (3264, (True, 'T', ('T', 1.0, 20, [('T', 20)]))),
 (3584, (True, 'C', ('C', 1.0, 38, [('C', 38)]))),
 (3904, (True, 'A', ('A', 1.0, 22, [('A', 22)]))),
 (4224, (True, 'G', ('G', 0.95, 19, [('G', 19), ('T', 1)]))),
 (4544, (True, 'C', ('C', 1.0, 27, [('C', 27)])))]

In [18]:
matches_confident = confident_matches.count()
matches_confident / total_basepairs

0.9838976302176584

In [19]:
matches = confident_matches.filter(lambda x: x[1][0]).count()
matches

4871331

In [20]:
matches / matches_confident

0.9999336980828277

In [21]:
mutations = confident_matches.filter(lambda x: not x[1][0])
mutations.take(20)

[(830720, (False, 'A', ('G', 0.7391304347826086, 17, [('G', 17), ('A', 6)]))),
 (832832, (False, 'C', ('T', 0.90625, 29, [('T', 29), ('C', 3)]))),
 (4857664, (False, 'G', ('T', 1.0, 31, [('T', 31)]))),
 (4857728, (False, 'A', ('C', 1.0, 21, [('C', 21)]))),
 (4857792, (False, 'G', ('A', 1.0, 32, [('A', 32)]))),
 (4857600, (False, 'G', ('C', 1.0, 30, [('C', 30)]))),
 (4857537, (False, 'A', ('C', 1.0, 21, [('C', 21)]))),
 (4857665, (False, 'C', ('T', 1.0, 31, [('T', 31)]))),
 (4857729, (False, 'C', ('G', 0.9523809523809523, 20, [('G', 20), ('C', 1)]))),
 (832769, (False, 'C', ('G', 0.75, 21, [('G', 21), ('C', 7)]))),
 (4857793, (False, 'G', ('A', 0.9375, 30, [('A', 30), ('G', 1), ('C', 1)]))),
 (4857601, (False, 'C', ('A', 1.0, 28, [('A', 28)]))),
 (830786, (False, 'A', ('G', 0.7391304347826086, 17, [('G', 17), ('A', 6)]))),
 (4857602, (False, 'A', ('G', 1.0, 28, [('G', 28)]))),
 (4857666, (False, 'C', ('T', 0.967741935483871, 30, [('T', 30), ('A', 1)]))),
 (4857730, (False, 'G', ('C', 1.

In [22]:
mutations.count()

323